In [1]:
import pandas as pd

import pickle

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

## Loading Data and Model From Disk

In [2]:
# Load the model from disk
MODEL_FILE = 'model.sav'
model = pickle.load(open(MODEL_FILE, 'rb'))

# Load dataframe from disk
college_data = pd.read_csv('data/college_data', delimiter=',')

In [4]:
def prepare_x(index):
    df = college_data.drop(columns=['Name', 'Rank'])
    x = df.loc[index]

    return x

In [5]:
# Will predict the ranking of the university at the second index in college_data
x = prepare_x(2)
model.predict([x])

array([53.63092778])